In [1]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor  # Import RandomForestRegressor

In [2]:
Variables_level_url = "https://raw.githubusercontent.com/Raziye-Aghapour/RET_GreenBuilding/main/RET%20-Variables_level.csv"
Factors_levels = pd.read_csv(Variables_level_url)

In [3]:
# classify our features between the ones that are fixed and the ones that will be
# part of the optimization problem

Cat_features=Factors_levels[Factors_levels['type'] == 'Categorical']['description'].tolist()

Num_features=Factors_levels[Factors_levels['type'] == 'Numerical-Continuous']['description'].tolist()


print(len(Cat_features))

10


In [4]:

def build_model_and_encoded_data(link, Cat_features, rf_params):
    # Read data from the provided link
    Cost = pd.read_excel(link, engine='openpyxl')
    features = Cost.columns.tolist()[:-1]
    target = Cost.columns.tolist()[-1]

    # Encode categorical variables
    column_transformer = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(sparse_output=False), Cat_features)
        ],
        remainder='passthrough'
    )
    transformed_data = column_transformer.fit_transform(Cost)
    new_columns = column_transformer.get_feature_names_out()
    encoded_Cost = pd.DataFrame(transformed_data, columns=new_columns)
    encoded_features = encoded_Cost.columns.tolist()[:-1]
    encoded_target = encoded_Cost.columns.tolist()[-1]

    # Split the data for training and testing
    X = encoded_Cost[encoded_features]
    y = encoded_Cost[encoded_target]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, train_size=0.8, random_state=1
    )

    # Build Random Forest Regressor model
    rf = RandomForestRegressor(**rf_params)  # Use RandomForestRegressor with rf_params
    rf_model = make_pipeline(rf)
    rf_model.fit(X_train, y_train)

    return rf_model, encoded_Cost



In [5]:
# Example usage:
DOE_url_Cost = "https://github.com/Raziye-Aghapour/RET_GreenBuilding/raw/main/RET%20-%20Cost%20Output.xlsx"
Cost = pd.read_excel(DOE_url_Cost, engine='openpyxl')

DOE_url_GWP = "https://github.com/Raziye-Aghapour/RET_GreenBuilding/raw/main/RET%20-%20GWP%20Output.xlsx"
GWP = pd.read_excel(DOE_url_GWP, engine='openpyxl')

DOE_url_HHP = "https://github.com/Raziye-Aghapour/RET_GreenBuilding/raw/main/RET%20-%20HHP%20Output.xlsx"
HHP = pd.read_excel(DOE_url_HHP, engine='openpyxl')

In [6]:
rf_params = {
    'n_estimators': 13,
    'max_depth': 3,
    'min_samples_split': 2,
    'min_samples_leaf': 1,
    'max_features': 'auto',
    'bootstrap': True,
    'random_state': 42
}


gbr_tree, encoded_Cost = build_model_and_encoded_data(DOE_url_Cost, Cat_features, rf_params)
gbr_tree_GWP, encoded_GWP = build_model_and_encoded_data(DOE_url_GWP, Cat_features, rf_params)
gbr_tree_HHP, encoded_HHP = build_model_and_encoded_data(DOE_url_HHP, Cat_features, rf_params)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors

In [7]:
##Grouping the categorical features


from collections import defaultdict

# Initialize a defaultdict to hold lists of one-hot encoded variables for each original category
one_hot_to_original_mapping = defaultdict(list)

# Assuming 'encoded_features' is a list of your one-hot encoded feature names
# and 'Cat_features' is a list of your original categorical feature names
for feature_name in encoded_Cost.columns.tolist()[:-1]:
    for cat_var in Cat_features:
        if cat_var in feature_name:
            one_hot_to_original_mapping[cat_var].append(feature_name)
            break  # Assuming each one-hot encoded feature can only belong to one category

# Now, one_hot_to_original_mapping maps from original categories to lists of one-hot encoded feature names
print(one_hot_to_original_mapping)

defaultdict(<class 'list'>, {'RoofExtFinish': ['cat__RoofExtFinish_Aluminum Paint', 'cat__RoofExtFinish_Dark', 'cat__RoofExtFinish_Light', 'cat__RoofExtFinish_Uncolored'], 'AGWExtFinish': ['cat__AGWExtFinish_Dark', 'cat__AGWExtFinish_Light', 'cat__AGWExtFinish_Medium', 'cat__AGWExtFinish_Uncolored'], 'GlassCategory': ['cat__GlassCategory_Double Low E', 'cat__GlassCategory_Quadruple Low E', 'cat__GlassCategory_Single Low E', 'cat__GlassCategory_Triple Low E'], 'GlassTypeEmissivity': ['cat__GlassTypeEmissivity_High', 'cat__GlassTypeEmissivity_Low'], 'FrameType': ['cat__FrameType_AlumwBrkFixedMtlSpacer', 'cat__FrameType_FiberglassFixedMlt spacer', 'cat__FrameType_ReinforcedVinylFixedMtlSpacer', 'cat__FrameType_WoofFixedMltSpacer'], 'System1HeatingSource': ['cat__System1HeatingSource_DX Coils', 'cat__System1HeatingSource_Electric Resistance'], 'System1SystemType': ['cat__System1SystemType_Packaged VVT', 'cat__System1SystemType_Split System Single Zone'], 'SupplyFans': ['cat__SupplyFans_For

In [8]:
%pip install gurobipy
%pip install gurobipy_pandas
%pip install gurobi-machinelearning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.8/70.8 kB 2.1 MB/s eta 0:00:00


In [9]:
from gurobi_ml import add_predictor_constr
import gurobipy as gp
import gurobipy_pandas as gppd

In [10]:
#Create an environment with your WLS license
params = {
'WLSACCESSID':'99f58439-b13c-43ba-b87a-192446a35271',
'WLSSECRET':'f24b1d50-561e-4794-aa2c-6633b350529c',
'LICENSEID':2494688
}

env = gp.Env(params=params)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2494688
Academic license 2494688 - for non-commercial use only - registered to rx___@mavs.uta.edu


In [11]:
# Define the model
m = gp.Model(env=env)

In [12]:
# Add binary variables for the one-hot encoded columns
binary_columns = [col for col in encoded_Cost.columns if col.startswith('cat__')]
for column in binary_columns:
    m.addVar(vtype=gp.GRB.BINARY, name=column)

# Add continuous variables for the original numerical variables
# We use the prefix 'remainder__' to identify the original numerical columns
continuous_columns = [col for col in encoded_Cost[encoded_Cost.columns.tolist()[:-1]] if col.startswith('remainder__')]
for column in continuous_columns:
    # Extract the original column name (without 'remainder__' prefix)
    original_column_name = column.replace('remainder__', '')
    # Define bounds based on the min and max values of the original column
    lb = Cost[original_column_name].min()
    ub = Cost[original_column_name].max()
    # Add the variable to the model with the defined bounds
    m.addVar(lb=lb, ub=ub, vtype=gp.GRB.CONTINUOUS, name=column)

# Update the model to integrate the new variables
# Now, your model has the necessary decision variables defined as per your dataset.
m.update()

In [13]:
# Assuming 'm' is your Gurobi model
for cat_var, one_hot_vars in one_hot_to_original_mapping.items():
    # Sum the binary variables corresponding to the one-hot encoded categories of each original feature
    # Ensure that this sum is >= 1, meaning at least one category must be selected
    m.addConstr(gp.quicksum(m.getVarByName(var) for var in one_hot_vars) >= 1, name=f"select_at_least_one_{cat_var}")

# Update the model to integrate the new constraints
m.update()

In [14]:
Cost_approx = m.addVar(name="Cost_approx")
GWP_approx = m.addVar(name="GWP_approx")
HHP_approx = m.addVar(name="HHP_approx")

m.update()

In [15]:
m.setObjective((Cost_approx-3.576490274725e+03)/3.576490274725e+03+(GWP_approx-1.684663092799e+05)/1.684663092799e+05+(HHP_approx-4.852619352383e+02)/4.852619352383e+02, gp.GRB.MINIMIZE)

In [16]:
# Assuming m is your Gurobi model
variables_dict = {var.VarName: var for var in m.getVars()}
# Create a DataFrame with a single row containing the Gurobi variables
variables_df = pd.DataFrame(variables_dict, index=[0])

In [17]:
variables_df[encoded_Cost.columns.tolist()[:-1]]

,cat__RoofExtFinish_Aluminum Paint,cat__RoofExtFinish_Dark,cat__RoofExtFinish_Light,cat__RoofExtFinish_Uncolored,cat__AGWExtFinish_Dark,cat__AGWExtFinish_Light,cat__AGWExtFinish_Medium,cat__AGWExtFinish_Uncolored,cat__GlassCategory_Double Low E,cat__GlassCategory_Quadruple Low E,...,cat__HeaterType_instantaneous,remainder__RoofExteriorInsulation,remainder__RoofAddlInsulation,remainder__AGWExteriorInsulation,remainder__AGWInteriorInsulation,remainder__AGWAddlInsulation,remainder__CeilingsBattInsulation,remainder__VerticalWallsBattInsulation,remainder__GlassTypeThickness,remainder__GlassTypeSpacing
0,<gurobi.Var cat__RoofExtFinish_Aluminum Paint>,<gurobi.Var cat__RoofExtFinish_Dark>,<gurobi.Var cat__RoofExtFinish_Light>,<gurobi.Var cat__RoofExtFinish_Uncolored>,<gurobi.Var cat__AGWExtFinish_Dark>,<gurobi.Var cat__AGWExtFinish_Light>,<gurobi.Var cat__AGWExtFinish_Medium>,<gurobi.Var cat__AGWExtFinish_Uncolored>,<gurobi.Var cat__GlassCategory_Double Low E>,<gurobi.Var cat__GlassCategory_Quadruple Low E>,...,<gurobi.Var cat__HeaterType_instantaneous>,<gurobi.Var remainder__RoofExteriorInsulation>,<gurobi.Var remainder__RoofAddlInsulation>,<gurobi.Var remainder__AGWExteriorInsulation>,<gurobi.Var remainder__AGWInteriorInsulation>,<gurobi.Var remainder__AGWAddlInsulation>,<gurobi.Var remainder__CeilingsBattInsulation>,<gurobi.Var remainder__VerticalWallsBattInsula...,<gurobi.Var remainder__GlassTypeThickness>,<gurobi.Var remainder__GlassTypeSpacing>


In [18]:
from gurobi_ml import add_predictor_constr

# Assuming gbr_model is your trained gradient boosting model
# And assuming feats is a list of feature names used by your model, which should match the column names in variables_df

# Use add_predictor_constr to add your model to the optimization problem
pred_constr1 = add_predictor_constr(m, gbr_tree, variables_df[encoded_Cost.columns.tolist()[:-1]], Cost_approx)
pred_constr2 = add_predictor_constr(m, gbr_tree_GWP, variables_df[encoded_Cost.columns.tolist()[:-1]], GWP_approx)
pred_constr3 = add_predictor_constr(m, gbr_tree_HHP, variables_df[encoded_Cost.columns.tolist()[:-1]], HHP_approx)
# Update the model to integrate the new constraints
m.update()

pred_constr1.print_stats()

Model for pipe:
117 variables
14 constraints
416 general constraints
Input has shape (1, 38)
Output has shape (1, 1)

Pipeline has 1 steps:

--------------------------------------------------------------------------------
Step            Output Shape    Variables              Constraints              
                                                Linear    Quadratic      General
rand_forest_reg         (1, 1)          117           14            0          416

--------------------------------------------------------------------------------


In [19]:
m.write('boosted_tree_opt.lp')


In [ ]:
m.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Academic license 2494688 - for non-commercial use only - registered to rx___@mavs.uta.edu
Optimize a model with 52 rows, 392 columns and 383 nonzeros
Model fingerprint: 0xc1607867
Model has 1248 general constraints
Variable types: 51 continuous, 341 integer (341 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [6e-06, 2e-03]
  Bounds range     [1e-01, 6e+01]
  RHS range        [1e+00, 1e+00]
  GenCon rhs range [2e-01, 3e+05]
  GenCon coe range [1e+00, 1e+00]
Presolve added 663 rows and 255 columns
Presolve time: 0.08s
Presolved: 715 rows, 647 columns, 2479 nonzeros
Presolved model has 312 SOS constraint(s)
Variable types: 315 continuous, 332 integer (330 binary)
Found heuristic solution: objective 0.9091496
Found heuris

In [ ]:
# Variable info
varInfo = [(v.varName, v.X, v.LB, v.UB) for v in m.getVars() ]
df = pd.DataFrame(varInfo)
df.columns=['Variable Name','Solution Value', 'LB','UB']
df.to_excel("variables.xlsx", index=False)
df_var = df.query('`Solution Value` > 0')
df.to_csv('out.csv')
df_var

In [ ]:
m.dispose()